In [1]:
# Importing the necessary libraries:
import openai
import os
from dotenv import load_dotenv
from datasets import load_dataset

c:\Users\ravit\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# ------NOTE: Use this piece of code when you are running the code on you local machine ##-------
# Loading the API key into the environment:
load_dotenv('D:/Learning/Gen AI/Building LLM Applications using Prompt Engineering/API Keys/.env')

# Assigning the API key to a variable:
OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]

# Assigning client:
client = openai.OpenAI()

# ## ------NOTE: Use this piece of code when you are running the code on Google colab (Assign the API key in the secrets tab on the left)##-------
# from google.colab import userdata
# # Assigning the API key to a variable:
# OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
# openai.api_key = OPENAI_API_KEY
# client = openai

In [3]:
# OpenAI function parameters:
model_1 ='gpt-3.5-turbo'
model_2 ='gpt-4-turbo'

In [4]:
# Function accepts the prompt as an input and returns the response
def get_response(prompt, sys_prompt="You are a helpful assistant", temperature=0, model="gpt-3.5-turbo"):
    messages = [{"role":"system","content":sys_prompt},
                {"role":"user","content":prompt}
                ]
    response = openai.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature
     )
    return response.choices[0].message.content

In [6]:
# Simple Prompt:
prompt = "Write a short blog post on God of War ragnarok"
print(get_response(prompt,temperature=0,model=model_2))

### Exploring the Epic Saga of God of War: Ragnarok

The release of "God of War: Ragnarok" has been one of the most anticipated events in the gaming world, following the critically acclaimed reboot of the series in 2018. Developed by Santa Monica Studio and published by Sony Interactive Entertainment, this sequel not only continues the poignant journey of Kratos and his son Atreus but also dives deeper into the rich tapestry of Norse mythology.

#### A Deeper Dive into Norse Lore

"God of War: Ragnarok" picks up where the previous game left off, with the prophecy of Ragnarok - the end of days in Norse mythology - hanging heavily over the protagonists. The game enriches its narrative by exploring more complex themes and delving into the conflicts and struggles of its characters. Players are taken on a journey through various realms of Norse mythology, each beautifully crafted and filled with new challenges and enemies.

#### Enhanced Gameplay and Mechanics

The gameplay in "Ragnarok" ha

In [5]:
# Loading the Huggingface dataset:
dataset = load_dataset("bigbio/med_qa")

c:\Users\ravit\AppData\Local\Programs\Python\Python312\Lib\site-packages\datasets\load.py:1486: FutureWarning: The repository for bigbio/med_qa contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/bigbio/med_qa
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


In [6]:
dataset

DatasetDict({
    train: Dataset({
        features: ['meta_info', 'question', 'answer_idx', 'answer', 'options'],
        num_rows: 10178
    })
    test: Dataset({
        features: ['meta_info', 'question', 'answer_idx', 'answer', 'options'],
        num_rows: 1273
    })
    validation: Dataset({
        features: ['meta_info', 'question', 'answer_idx', 'answer', 'options'],
        num_rows: 1272
    })
})

In [7]:
# Creating lists to hold the questions, options, answer_ids and answers
questions = []
answers = []

# Extracting the details from the trainin dataset into their respective lists:
for i in range(5):

  question = dataset['validation'][i]['question']
  options = dataset['validation'][i]['options']
  answer_id = dataset['validation'][i]['answer_idx']
  answer = dataset['validation'][i]['answer']

  # Creating List of options in text format:
  options_new = ''
  for i in range(len(options)):
    options_new = options_new +'('+options[i].get('key') +')'+options[i].get('value')+'\n'

  # Concatenating question to the list of options:
  question_new = question+'\n'+options_new
  # Concatenating answer_id and answer:
  answer_new = '('+answer_id+')'+answer

  # Adding question+options to the questions list:
  questions.append(question_new)
  # Adding answer_id+answer to the answers list:
  answers.append(answer_new)

In [8]:
# Question to test the LLM model:
validation_question = dataset['validation'][0]['question']
validation_options = dataset['validation'][0]['options']
validation_answer_id = dataset['validation'][0]['answer_idx']
validation_answer = dataset['validation'][0]['answer']

# Creating List of options in text format:
validation_options_new = ''
for i in range(len(validation_options)):
  validation_options_new = validation_options_new +'('+validation_options[i].get('key') +')'+validation_options[i].get('value')+'\n'

# Concatenating question to the list of options:
validation_question_new = validation_question+'\n'+validation_options_new
# Concatenating answer_id and answer:
validation_answer_new = '('+validation_answer_id+')'+validation_answer

# Displaying validation question and answer:
print(validation_question_new)
print(validation_answer_new)

A 21-year-old sexually active male complains of fever, pain during urination, and inflammation and pain in the right knee. A culture of the joint fluid shows a bacteria that does not ferment maltose and has no polysaccharide capsule. The physician orders antibiotic therapy for the patient. The mechanism of action of action of the medication given blocks cell wall synthesis, which of the following was given?
(A)Chloramphenicol
(B)Gentamicin
(C)Ciprofloxacin
(D)Ceftriaxone
(E)Trimethoprim

(D)Ceftriaxone


## **Few shot prompting:**

In [9]:
# Few shot prompting:
prompt = f'''Question:{questions[0]}
Answer:{answers[0]}

Question:{questions[1]}
Answer:{answers[1]}

Question:{questions[2]}
Answer:{answers[2]}

Question:{validation_question_new}
Answer:
'''
sys_prompt = "You are a helpful assistant that answers multiple choice questions about Medical knowledge"

# Response from gpt-3.5:
model_1_few_shot_response = get_response(prompt,sys_prompt,0,model_1)
print("GPT-3.5 response:")
print(model_1_few_shot_response)
print('\n')
# Response from gpt-4:
model_2_few_shot_response = get_response(prompt,sys_prompt,0,model_2)
print("GPT-4 response:")
print(model_2_few_shot_response)

GPT-3.5 response:
(D)Ceftriaxone


GPT-4 response:
(D)Ceftriaxone


## **One shot prompting:**

In [10]:
# one shot prompting:
prompt = f'''Question:{questions[0]}
Answer:{answers[0]}

Question:{validation_question_new}
Answer:
'''
sys_prompt = "You are a helpful assistant that answers multiple choice questions about Medical knowledge"

# Response from gpt-3.5:
model_1_one_shot_response = get_response(prompt,sys_prompt,0,model_1)
print("GPT-3.5 response:")
print(model_1_one_shot_response)
print('\n')
# Response from gpt-4:
model_2_one_shot_response = get_response(prompt,sys_prompt,0,model_2)
print("GPT-4 response:")
print(model_2_one_shot_response)

GPT-3.5 response:
(D)Ceftriaxone


GPT-4 response:
(D) Ceftriaxone

Ceftriaxone is a cephalosporin antibiotic that works by inhibiting bacterial cell wall synthesis, making it an appropriate choice for treating infections caused by bacteria that do not ferment maltose and lack a polysaccharide capsule, such as Neisseria gonorrhoeae.


## **Zero shot prompting:**

In [11]:
# Zero shot prompting:
prompt = f'''Question:{validation_question_new}
Answer:
'''
sys_prompt = "You are a helpful assistant that answers multiple choice questions about Medical knowledge"

# Response from gpt-3.5:
model_1_zero_shot_response = get_response(prompt,sys_prompt,0,model_1)
print("GPT-3.5 response:")
print(model_1_zero_shot_response)
print('\n')
# Response from gpt-4:
model_2_zero_shot_response = get_response(prompt,sys_prompt,0,model_2)
print("GPT-4 response:")
print(model_2_zero_shot_response)

GPT-3.5 response:
(D) Ceftriaxone

Ceftriaxone is a cephalosporin antibiotic that works by inhibiting cell wall synthesis in bacteria. It is commonly used to treat infections caused by gram-negative bacteria, such as gonorrhea and joint infections.


GPT-4 response:
The correct answer is (D) Ceftriaxone.

Ceftriaxone is a third-generation cephalosporin antibiotic that works by inhibiting bacterial cell wall synthesis. This makes it effective against a broad range of gram-positive and gram-negative bacteria. The symptoms described in the question, along with the culture results, suggest a bacterial infection that could be effectively treated with an antibiotic like ceftriaxone. The other options listed do not primarily function by blocking cell wall synthesis: Chloramphenicol inhibits protein synthesis, Gentamicin is an aminoglycoside that inhibits protein synthesis, Ciprofloxacin is a fluoroquinolone that inhibits DNA gyrase, and Trimethoprim is a folate synthesis inhibitor.
